<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/chat_bot_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version


In [ ]:
%cd /content

In [ ]:
!git clone https://github.com/weedge/chat-bot.git

In [ ]:
%cd /content/chat-bot

### install dependencies

In [ ]:
!apt-get install python3-pyaudio && pip -q install pyannote.audio

In [ ]:
!pip install -q pyaudio pydub redis

In [ ]:
# asr whisper
!pip install -q openai-whisper whisper-timestamped faster-whisper

In [ ]:
# asr SenseVoice with funasr
!pip install funasr

In [ ]:
# llamacpp https://github.com/abetlen/llama-cpp-python?tab=readme-ov-file#installation
# cpu
#!pip install llama-cpp-python
#!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cpu
#!CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS" pip install llama-cpp-python

# gpu
#!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python
!pip uninstall llama-cpp-python
!pip install llama-cpp-python \
  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122

In [ ]:
!# ChatTTS
!git submodule update --init --recursive
!pip install -q vocos
!pip install vector_quantize_pytorch

In [ ]:
# CoquiTTS
#!pip uninstall TTS
!pip install coqui-tts


In [ ]:
# CosyVoiceTTS
!git submodule update --init --recursive
!cd deps/CosyVoice && pip install -r requirements.txt


In [ ]:
!pip install numpy==1.25.0

In [ ]:
!pip list | grep torch

In [ ]:
!pip install accelerate optimum

In [ ]:
!pip show omegaconf einops IPython ctranslate2 numpy TTS transformers accelerate optimum llama-cpp-python coqui-tts

### download models

In [ ]:
# asr openai whisper ckpt
!wget https://openaipublic.azureedge.net/main/whisper/models/65147644a518d12f04e32d6f3b26facc3f8dd46e5390956a9424a650c0ce22b9/tiny.pt -O ./models/tiny.pt
!wget https://openaipublic.azureedge.net/main/whisper/models/ed3a0b6b1c0edf879ad9b11b1af5a0e6ab5db9205f891f668f8b0e6c6326e34e/base.pt -O ./models/base.pt
!wget https://openaipublic.azureedge.net/main/whisper/models/9ecf779972d90ba49c06d968637d720dd632c55bbf19d441fb42bf17a411e794/small.pt -O ./models/small.pt
!wget https://openaipublic.azureedge.net/main/whisper/models/345ae4da62f9b3d59415adc60127b97c714f32e89e936602e85993674d08dcb1/medium.pt -O ./models/medium.pt
!wget https://openaipublic.azureedge.net/main/whisper/models/e4b87e7e0bf463eb8e6956e646f1e277e901512310def2c24bf0e11bd3c28e9a/large-v1.pt -O ./models/large-v1.pt
!wget https://openaipublic.azureedge.net/main/whisper/models/81f7c96c852ee8fc832187b0132e569d6c3065a3252ed18e56effd0b6a73e524/large-v2.pt -O ./models/large-v2.pt
!wget https://openaipublic.azureedge.net/main/whisper/models/e5b1a55b89c1367dacf97e3e19bfd829a01529dbfdeefa8caeb59b3f1b81dadb/large-v3.pt -O ./models/large-v3.pt


In [ ]:
# asr hf openai whisper ckpt
!huggingface-cli download openai/whisper-base  --local-dir ./models/openai/whisper-base --local-dir-use-symlinks False
# faster-whisper
!huggingface-cli download Systran/faster-whisper-tiny  --local-dir ./models/Systran/faster-whisper-tiny --local-dir-use-symlinks False
!huggingface-cli download Systran/faster-whisper-base  --local-dir ./models/Systran/faster-whisper-base --local-dir-use-symlinks False
!huggingface-cli download Systran/faster-whisper-small  --local-dir ./models/Systran/faster-whisper-small --local-dir-use-symlinks False
!huggingface-cli download Systran/faster-whisper-medium  --local-dir ./models/Systran/faster-whisper-medium --local-dir-use-symlinks False
!huggingface-cli download Systran/faster-whisper-large-v1  --local-dir ./models/Systran/faster-whisper-large-v1 --local-dir-use-symlinks False
!huggingface-cli download Systran/faster-whisper-large-v2  --local-dir ./models/Systran/faster-whisper-large-v2 --local-dir-use-symlinks False
!huggingface-cli download Systran/faster-whisper-large-v3  --local-dir ./models/Systran/faster-whisper-large-v3 --local-dir-use-symlinks False
!huggingface-cli download Systran/faster-distil-whisper-large-v2  --local-dir ./models/Systran/faster-distil-whisper-large-v2 --local-dir-use-symlinks False
!huggingface-cli download Systran/faster-distil-whisper-large-v3  --local-dir ./models/Systran/faster-distil-whisper-large-v3 --local-dir-use-symlinks False

In [ ]:
# asr SenseVoice ckpt
!huggingface-cli download FunAudioLLM/SenseVoiceSmall  --local-dir ./models/FunAudioLLM/SenseVoiceSmall --local-dir-use-symlinks False

In [ ]:
# llm llamacpp phi-3-mini-4k-instruct
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-gguf Phi-3-mini-4k-instruct-q4.gguf --local-dir ./models --local-dir-use-symlinks False


In [ ]:
# llm llamacpp Qwen1.5-chat
!huggingface-cli download Qwen/Qwen1.5-7B-Chat-GGUF qwen1_5-7b-chat-q8_0.gguf  --local-dir ./models --local-dir-use-symlinks False
#!huggingface-cli download Qwen/Qwen1.5-14B-Chat-GGUF qwen1_5-14b-chat-q8_0.gguf  --local-dir ./models --local-dir-use-symlinks False


In [ ]:
!rm -f ./models/qwen2-57b-a14b-instruct-*

In [ ]:
# llm llamacpp Qwen2-Instruct
!huggingface-cli download Qwen/Qwen2-1.5B-Instruct-GGUF qwen2-1_5b-instruct-q8_0.gguf  --local-dir ./models --local-dir-use-symlinks False
!huggingface-cli download Qwen/Qwen2-1.5B-Instruct-GGUF qwen2-1_5b-instruct-fp16.gguf  --local-dir ./models --local-dir-use-symlinks False
!huggingface-cli download Qwen/Qwen2-7B-Instruct-GGUF qwen2-7b-instruct-q8_0.gguf  --local-dir ./models --local-dir-use-symlinks False
!huggingface-cli download Qwen/Qwen2-7B-Instruct-GGUF qwen2-7b-instruct-fp16.gguf  --local-dir ./models --local-dir-use-symlinks False
#!huggingface-cli download Qwen/Qwen2-57B-A14B-Instruct-GGUF qwen2-57b-a14b-instruct-q3_k_m.gguf  --local-dir ./models --local-dir-use-symlinks False
#!huggingface-cli download Qwen/Qwen2-57B-A14B-Instruct-GGUF --include qwen2-57b-a14b-instruct-q8_0-*.gguf  --local-dir ./models --local-dir-use-symlinks False


In [ ]:
#/content/llamacpp/llama-gguf-split --merge qwen2-57b-a14b-instruct-q8_0-00001-of-00002.gguf qwen2-57b-a14b-instruct-q8_0.gguf

In [ ]:
# llm chatglmcpp chatGLM

In [ ]:
# tts chatTTS
!huggingface-cli download 2Noise/ChatTTS  --local-dir ./models/2Noise/ChatTTS --local-dir-use-symlinks False

In [ ]:
# tts coquiTTS
!huggingface-cli download coqui/XTTS-v2  --local-dir ./models/coqui/XTTS-v2 --local-dir-use-symlinks False

In [ ]:
# tts cosy voice
!git lfs install
!mkdir -p models
!git clone https://www.modelscope.cn/iic/CosyVoice-300M.git models/CosyVoice-300M
!git clone https://www.modelscope.cn/iic/CosyVoice-300M-SFT.git models/CosyVoice-300M-SFT
!git clone https://www.modelscope.cn/iic/CosyVoice-300M-Instruct.git models/CosyVoice-300M-Instruct
!git clone https://www.modelscope.cn/iic/CosyVoice-ttsfrd.git models/CosyVoice-ttsfrd


In [ ]:
!cd models/CosyVoice-ttsfrd/ \
  && wget https://www.modelscope.cn/models/speech_tts/speech_kantts_ttsfrd/resolve/master/ttsfrd-0.3.9-cp310-cp310-linux_x86_64.whl
!cd models/CosyVoice-ttsfrd/ && unzip resource.zip -d .
!pip install models/CosyVoice-ttsfrd/ttsfrd-0.3.9-cp310-cp310-linux_x86_64.whl


### test


#### ASR

In [ ]:
!ASR_TAG=whisper_asr MODEL_NAME_OR_PATH=tiny \
  python -m unittest test.modules.speech.asr.test_whisper_asr.TestWhisperASR.test_transcribe

!ASR_TAG=whisper_asr MODEL_NAME_OR_PATH=base \
  python -m unittest test.modules.speech.asr.test_whisper_asr.TestWhisperASR.test_transcribe

!ASR_TAG=whisper_asr MODEL_NAME_OR_PATH=small \
  python -m unittest test.modules.speech.asr.test_whisper_asr.TestWhisperASR.test_transcribe

!ASR_TAG=whisper_asr MODEL_NAME_OR_PATH=medium \
  python -m unittest test.modules.speech.asr.test_whisper_asr.TestWhisperASR.test_transcribe

!ASR_TAG=whisper_asr MODEL_NAME_OR_PATH=large-v1 \
  python -m unittest test.modules.speech.asr.test_whisper_asr.TestWhisperASR.test_transcribe

!ASR_TAG=whisper_asr MODEL_NAME_OR_PATH=large-v2 \
  python -m unittest test.modules.speech.asr.test_whisper_asr.TestWhisperASR.test_transcribe

!ASR_TAG=whisper_asr MODEL_NAME_OR_PATH=large-v3 \
  python -m unittest test.modules.speech.asr.test_whisper_asr.TestWhisperASR.test_transcribe


In [ ]:
!ASR_TAG=whisper_transformers_asr MODEL_NAME_OR_PATH=./models/openai/whisper-base \
  python -m unittest test.modules.speech.asr.test_whisper_asr.TestWhisperASR.test_transcribe

In [ ]:
!MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-tiny ASR_VERBOSE=False ASR_TAG=whisper_faster_asr \
  python -m unittest test.modules.speech.asr.test_whisper_asr.TestWhisperASR.test_transcribe

!MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-base ASR_VERBOSE=False ASR_TAG=whisper_faster_asr \
  python -m unittest test.modules.speech.asr.test_whisper_asr.TestWhisperASR.test_transcribe

!MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-small ASR_VERBOSE=False ASR_TAG=whisper_faster_asr \
  python -m unittest test.modules.speech.asr.test_whisper_asr.TestWhisperASR.test_transcribe

!MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-medium ASR_VERBOSE=False ASR_TAG=whisper_faster_asr \
  python -m unittest test.modules.speech.asr.test_whisper_asr.TestWhisperASR.test_transcribe

!MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-large-v1 ASR_VERBOSE=False ASR_TAG=whisper_faster_asr \
  python -m unittest test.modules.speech.asr.test_whisper_asr.TestWhisperASR.test_transcribe

!MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-large-v2 ASR_VERBOSE=False ASR_TAG=whisper_faster_asr \
  python -m unittest test.modules.speech.asr.test_whisper_asr.TestWhisperASR.test_transcribe

!MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-large-v3 ASR_VERBOSE=False ASR_TAG=whisper_faster_asr \
  python -m unittest test.modules.speech.asr.test_whisper_asr.TestWhisperASR.test_transcribe

#### llm

In [ ]:
!MODEL_TYPE=generate  python -m unittest test.core.llm.test_llamacpp.TestLLamacppLLM.test_generate

In [ ]:
!MODEL_TYPE=generate N_GPU_LAYERS=29 FLASH_ATTN=1 python -m unittest test.core.llm.test_llamacpp.TestLLamacppLLM.test_generate

In [ ]:
!MODEL_TYPE=generate N_GPU_LAYERS=33 FLASH_ATTN=1 \
  MODEL_PATH=./models/qwen1_5-7b-chat-q8_0.gguf \
  python -m unittest test.core.llm.test_llamacpp.TestLLamacppLLM.test_generate

In [ ]:
!MODEL_TYPE=generate N_GPU_LAYERS=33 FLASH_ATTN=1 \
  MODEL_PATH=./models/qwen1_5-14b-chat-q8_0.gguf \
  python -m unittest test.core.llm.test_llamacpp.TestLLamacppLLM.test_generate

In [ ]:
!MODEL_TYPE=generate N_GPU_LAYERS=29 FLASH_ATTN=1 \
  MODEL_PATH=./models/qwen2-1_5b-instruct-fp16.gguf \
  python -m unittest test.core.llm.test_llamacpp.TestLLamacppLLM.test_generate

In [ ]:
!MODEL_TYPE=generate \
  MODEL_PATH=./models/qwen2-7b-instruct-q8_0.gguf \
  python -m unittest test.core.llm.test_llamacpp.TestLLamacppLLM.test_generate

In [ ]:
!MODEL_TYPE=generate \
  MODEL_PATH=./models/qwen2-7b-instruct-fp16.gguf \
  python -m unittest test.core.llm.test_llamacpp.TestLLamacppLLM.test_generate

In [ ]:
!MODEL_TYPE=generate \
  MODEL_PATH=./models/qwen2-7b-instruct-fp16.gguf \
  python -m unittest test.core.llm.test_llamacpp.TestLLamacppLLM.test_generate


#### tts

In [ ]:
#!python -m unittest test.modules.speech.tts.test_cosy_voice.TestCosyVoiceTTS.test_get_voices
!python -m unittest test.modules.speech.tts.test_cosy_voice.TestCosyVoiceTTS.test_synthesize

In [ ]:
from IPython.display import Audio
Audio("./records/test_tts_cosy_voice.wav", autoplay=True)


In [ ]:
Audio("./records/tts_speech.wav", autoplay=True)


### run chat bot BE

In [ ]:
from google.colab import userdata
redis_pwd = userdata.get('REDIS_PASSWORD')

In [ ]:
!mkdir -p log

In [ ]:
!REDIS_PASSWORD=$redis_pwd RUN_OP=be TQDM_DISABLE=True python -m src.cmd.remote-queue-chat.generate_audio2audio > ./log/be_std_out.log

In [ ]:
!ASR_TAG=whisper_faster_asr \
  ASR_MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-base \
  REDIS_PASSWORD=$redis_pwd \
  RUN_OP=be \
  TQDM_DISABLE=True \
  python -m src.cmd.remote-queue-chat.generate_audio2audio > ./log/be_std_out.log

In [ ]:
!ASR_TAG=whisper_faster_asr \
  ASR_MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-base \
  REDIS_PASSWORD=$redis_pwd \
  RUN_OP=be \
  TQDM_DISABLE=True \
  TTS_TAG=tts_coqui \
  N_GPU_LAYERS=33 FLASH_ATTN=1 \
  LLM_MODEL_NAME=qwen \
  LLM_MODEL_PATH=./models/qwen1_5-7b-chat-q8_0.gguf \
  python -m src.cmd.remote-queue-chat.generate_audio2audio > ./log/be_std_out.log

In [ ]:
!ASR_TAG=whisper_faster_asr \
  ASR_MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-base \
  REDIS_PASSWORD=$redis_pwd \
  RUN_OP=be \
  TQDM_DISABLE=True \
  TTS_TAG=tts_coqui TTS_STREAM=1 \
  N_GPU_LAYERS=33 FLASH_ATTN=1 \
  LLM_MODEL_NAME=qwen \
  LLM_MODEL_PATH=./models/qwen1_5-7b-chat-q8_0.gguf \
  python -m src.cmd.remote-queue-chat.generate_audio2audio > ./log/be_std_out.log

In [ ]:
!ASR_TAG=whisper_faster_asr \
  ASR_MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-base \
  REDIS_PASSWORD=$redis_pwd \
  RUN_OP=be \
  TQDM_DISABLE=True \
  TTS_TAG=tts_coqui \
  N_GPU_LAYERS=33 FLASH_ATTN=1 \
  LLM_MODEL_NAME=qwen \
  LLM_MODEL_PATH=./models/qwen1_5-14b-chat-q8_0.gguf \
  python -m src.cmd.remote-queue-chat.generate_audio2audio > ./log/be_std_out.log

In [ ]:
!ASR_TAG=whisper_faster_asr \
  ASR_MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-base \
  REDIS_PASSWORD=$redis_pwd \
  RUN_OP=be \
  TQDM_DISABLE=True \
  TTS_TAG=tts_coqui \
  N_GPU_LAYERS=29 FLASH_ATTN=1 \
  LLM_MODEL_NAME=qwen \
  LLM_MODEL_PATH=./models/qwen2-1_5b-instruct-q8_0.gguf \
  python -m src.cmd.remote-queue-chat.generate_audio2audio > ./log/be_std_out.log

In [ ]:
!ASR_TAG=whisper_faster_asr \
  ASR_MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-base \
  REDIS_PASSWORD=$redis_pwd \
  RUN_OP=be \
  TQDM_DISABLE=True \
  TTS_TAG=tts_coqui \
  N_GPU_LAYERS=29 FLASH_ATTN=1 \
  LLM_MODEL_NAME=qwen \
  LLM_MODEL_PATH=./models/qwen2-1_5b-instruct-fp16.gguf \
  python -m src.cmd.remote-queue-chat.generate_audio2audio > ./log/be_std_out.log

In [ ]:
!ASR_TAG=whisper_faster_asr \
  ASR_MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-base \
  REDIS_PASSWORD=$redis_pwd \
  RUN_OP=be \
  TQDM_DISABLE=True \
  TTS_TAG=tts_coqui \
  LLM_MODEL_NAME=qwen \
  LLM_MODEL_PATH=./models/qwen2-7b-instruct-q8_0.gguf \
  python -m src.cmd.remote-queue-chat.generate_audio2audio > ./log/be_std_out.log

In [ ]:
!ASR_TAG=whisper_faster_asr \
  ASR_MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-base \
  REDIS_PASSWORD=$redis_pwd \
  RUN_OP=be \
  TQDM_DISABLE=True \
  TTS_TAG=tts_coqui \
  LLM_MODEL_NAME=qwen \
  LLM_MODEL_PATH=./models/qwen2-7b-instruct-fp16.gguf \
  python -m src.cmd.remote-queue-chat.generate_audio2audio > ./log/be_std_out.log

In [ ]:
# qwen2-57b moe
!ASR_TAG=whisper_faster_asr \
  ASR_MODEL_NAME_OR_PATH=./models/Systran/faster-whisper-base \
  REDIS_PASSWORD=$redis_pwd \
  RUN_OP=be \
  TQDM_DISABLE=True \
  TTS_TAG=tts_coqui \
  LLM_MODEL_NAME=qwen \
  LLM_MODEL_PATH=./models/qwen2-57b-a14b-instruct-q3_k_m.gguf \
  python -m src.cmd.remote-queue-chat.generate_audio2audio > ./log/be_std_out.log

In [ ]:
!RUN_OP=be \
  TQDM_DISABLE=True \
  REDIS_PASSWORD=$redis_pwd \
  ASR_TAG=sense_voice_asr \
  ASR_LANG=zn \
  N_GPU_LAYERS=33 FLASH_ATTN=1 \
  LLM_MODEL_NAME=qwen \
  LLM_MODEL_PATH=./models/qwen1_5-7b-chat-q8_0.gguf \
  ASR_MODEL_NAME_OR_PATH=./models/FunAudioLLM/SenseVoiceSmall \
  TTS_TAG=tts_cosy_voice \
  python -m src.cmd.remote-queue-chat.generate_audio2audio > ./log/be_std_out.log

### opt

todo:
- batch stream tts
- use more gpu memory to parallel computing for model tensor weight
- sft asr whisper model, llm, tts gen model
